In [1]:
import psycopg as pg
import password

In [2]:
conn = pg.connect(dbname="postgres", host="localhost", user="postgres", password=password.password, port=5432)
cursor = conn.cursor()

## Список запросов к базе данных

In [3]:
requests = [
"""
/* Выводим все уникальные бренды, у которых стандартная 
 * стоимость выше 1500 долларов. */
SELECT DISTINCT brand
FROM "transaction" t
WHERE standard_cost > 1500;
""",
"""
/* Выводим все подтвержденные транзакции за период 
 * '2017-04-01' по '2017-04-09' включительно 
 * Под транзакцией понимаем transaction_id. Если нужно вывести
 * строки соответствующие этому transaction_id, то указываем SELECT * 
 */
SELECT transaction_id
FROM "transaction" t
WHERE TO_DATE(transaction_date, 'DD.MM.YYYY') BETWEEN '2017-04-01' AND '2017-04-09'
  AND order_status = 'Approved';
""",
"""
/* Выводим все профессии у клиентов из сферы IT или 
 * Financial Services, которые начинаются с фразы 'Senior'.
 * Под 'всеми профессиями' понимаем уникальные должности, начинающиеся
 * с 'Senior' в указанных отраслях.*/
SELECT DISTINCT job_title
FROM "customer" c
WHERE job_industry_category IN ('IT', 'Financial Services') AND job_title LIKE 'Senior%';
""",
"""
/* Выводим все бренды, которые закупают клиенты, 
 * работающие в сфере Financial Services */
SELECT DISTINCT t.brand
FROM "transaction" t 
JOIN customer c ON t.customer_id = c.customer_id
WHERE c.job_industry_category = 'Financial Services';
""",
"""
/* Выводим 10 клиентов, которые оформили онлайн-заказ 
 * продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'. 
 * Исходим из того, что под клиентом понимается customer_id. Если для описания
 * клиента понадобятся атрибуты из таблицы customer, то их можно вывести,
 * объединив таблицы*/
SELECT customer_id
FROM "transaction" t 
WHERE brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles') AND online_order = 'True'
LIMIT 10;
""",
"""
/* Выводим всех клиентов, у которых нет транзакций. 
 * Исходим из того, что под клиентом понимается customer_id. Если под клиентом
 * будет пониматься сочетание first_name и last_name, то их можно вывести,
 * добавив в SELECT c.first_name, c.last_name. */
SELECT c.customer_id
FROM "customer" c 
LEFT JOIN "transaction" t ON c.customer_id = t.customer_id
WHERE t.customer_id IS NULL;
""",
"""
/* Выводим всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.
 * Исходим из того, что под клиентом понимается customer_id. При необходимости
 * иные поля из таблицы customer можно вывести, добавив их в SELECT с
 * использованием alias 'с'. Под 'всех' в контексте задания
 * понимаем всех уникальных клиентов*/
SELECT DISTINCT c.customer_id
FROM "customer" c 
JOIN "transaction" t ON t.customer_id = c.customer_id 
WHERE c.job_industry_category = 'IT' AND t.standard_cost = (
	SELECT MAX(standard_cost) FROM "transaction");
""",
"""
/* Выводим всех клиентов из сферы IT и Health, у которых есть подтвержденные 
 * транзакции за период '2017-07-07' по '2017-07-17'
 * Исходим из того, что под клиентом понимается customer_id. При необходимости
 * иные поля из таблицы customer можно вывести, добавив их в SELECT с
 * использованием alias 'с'. Под 'всех' в контексте задания
 * понимаем всех уникальных клиентов*/
SELECT DISTINCT t.customer_id
FROM "transaction" t 
JOIN customer c ON t.customer_id = c.customer_id 
WHERE c.job_industry_category IN ('IT', 'Health') AND
(TO_DATE(transaction_date, 'DD.MM.YYYY')
	BETWEEN '2017-07-07' AND '2017-07-17')
"""
]

In [4]:
def request(n: int):
    r = requests[n]
    print(r)
    cursor.execute(r)
    answer = cursor.fetchall()
    for i in answer:
        print(*i)

### Задание 1. Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов

In [5]:
request(0)


/* Выводим все уникальные бренды, у которых стандартная 
 * стоимость выше 1500 долларов. */
SELECT DISTINCT brand
FROM "transaction" t
WHERE standard_cost > 1500;

OHM Cycles
WeareA2B
Trek Bicycles
Solex
Norco Bicycles
Giant Bicycles


### Задание 2. Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно

In [6]:
request(1)


/* Выводим все подтвержденные транзакции за период 
 * '2017-04-01' по '2017-04-09' включительно 
 * Под транзакцией понимаем transaction_id. Если нужно вывести
 * строки соответствующие этому transaction_id, то указываем SELECT * 
 */
SELECT transaction_id
FROM "transaction" t
WHERE TO_DATE(transaction_date, 'DD.MM.YYYY') BETWEEN '2017-04-01' AND '2017-04-09'
  AND order_status = 'Approved';

17
19
23
83
89
126
146
154
220
228
280
290
304
526
534
563
669
671
767
898
904
943
948
1044
1049
1073
1074
1125
1161
1187
1197
1235
1242
1254
1275
1294
1342
1418
1497
1503
1593
1639
1653
1686
1695
1707
1714
1799
1825
1827
1854
1890
1911
1930
1999
2001
2010
2049
2051
2055
2070
2099
2146
2222
2251
2277
2304
2370
2383
2410
2420
2451
2460
2508
2529
2551
2564
2590
2638
2660
2677
2694
2724
2810
2817
2913
2940
2988
3042
3189
3214
3247
3363
3382
3440
3457
3459
3483
3510
3524
3527
3551
3560
3588
3611
3691
3707
3716
3717
3765
3779
3819
3822
3972
4069
4073
4074
4118
4137
4316
4340
4386
4416
4452
4458
4477


### Задание 3. Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'

In [7]:
request(2)


/* Выводим все профессии у клиентов из сферы IT или 
 * Financial Services, которые начинаются с фразы 'Senior'.
 * Под 'всеми профессиями' понимаем уникальные должности, начинающиеся
 * с 'Senior' в указанных отраслях.*/
SELECT DISTINCT job_title
FROM "customer" c
WHERE job_industry_category IN ('IT', 'Financial Services') AND job_title LIKE 'Senior%';

Senior Cost Accountant
Senior Developer
Senior Editor
Senior Financial Analyst
Senior Quality Engineer
Senior Sales Associate


### Задание 4. Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services

In [8]:
request(3)


/* Выводим все бренды, которые закупают клиенты, 
 * работающие в сфере Financial Services */
SELECT DISTINCT t.brand
FROM "transaction" t 
JOIN customer c ON t.customer_id = c.customer_id
WHERE c.job_industry_category = 'Financial Services';

None
Trek Bicycles
WeareA2B
Solex
Giant Bicycles
OHM Cycles
Norco Bicycles


### Задание 5. Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'

In [9]:
request(4)


/* Выводим 10 клиентов, которые оформили онлайн-заказ 
 * продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'. 
 * Исходим из того, что под клиентом понимается customer_id. Если для описания
 * клиента понадобятся атрибуты из таблицы customer, то их можно вывести,
 * объединив таблицы*/
SELECT customer_id
FROM "transaction" t 
WHERE brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles') AND online_order = 'True'
LIMIT 10;

3120
787
2339
1243
3002
2666
3368
2810
2003
2448


### Задание 6. Вывести всех клиентов, у которых нет транзакций

In [10]:
request(5)


/* Выводим всех клиентов, у которых нет транзакций. 
 * Исходим из того, что под клиентом понимается customer_id. Если под клиентом
 * будет пониматься сочетание first_name и last_name, то их можно вывести,
 * добавив в SELECT c.first_name, c.last_name. */
SELECT c.customer_id
FROM "customer" c 
LEFT JOIN "transaction" t ON c.customer_id = t.customer_id
WHERE t.customer_id IS NULL;

852
869
1373
2074
2660
3229
3236
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616


### Задание 7. Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью

In [11]:
request(6)


/* Выводим всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.
 * Исходим из того, что под клиентом понимается customer_id. При необходимости
 * иные поля из таблицы customer можно вывести, добавив их в SELECT с
 * использованием alias 'с'. Под 'всех' в контексте задания
 * понимаем всех уникальных клиентов*/
SELECT DISTINCT c.customer_id
FROM "customer" c 
JOIN "transaction" t ON t.customer_id = c.customer_id 
WHERE c.job_industry_category = 'IT' AND t.standard_cost = (
	SELECT MAX(standard_cost) FROM "transaction");

34
893
975
1672
1773
1918
2913
3151
3473


### Задание 8. Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'

In [12]:
request(7)


/* Выводим всех клиентов из сферы IT и Health, у которых есть подтвержденные 
 * транзакции за период '2017-07-07' по '2017-07-17'
 * Исходим из того, что под клиентом понимается customer_id. При необходимости
 * иные поля из таблицы customer можно вывести, добавив их в SELECT с
 * использованием alias 'с'. Под 'всех' в контексте задания
 * понимаем всех уникальных клиентов*/
SELECT DISTINCT t.customer_id
FROM "transaction" t 
JOIN customer c ON t.customer_id = c.customer_id 
WHERE c.job_industry_category IN ('IT', 'Health') AND
(TO_DATE(transaction_date, 'DD.MM.YYYY')
	BETWEEN '2017-07-07' AND '2017-07-17')

22
28
41
47
104
235
239
249
289
290
300
320
331
338
350
382
414
424
428
503
526
579
590
612
661
679
682
685
694
708
712
722
753
759
769
779
788
872
886
892
893
936
949
950
973
1005
1024
1044
1076
1091
1095
1166
1174
1189
1231
1237
1328
1337
1343
1376
1394
1489
1516
1538
1542
1555
1578
1585
1683
1693
1760
1836
1862
1907
1929
1952
1956
2000
2024
2205
2210
2235
2285
2317
2337
2341
2